# Assignment Customer Satisfaction and Sentiment Analysis


## Objective

You are a data analyst of a consulting company that provides customer insight regarding multiple ticketing system, such as JIRA and Zoho Desk. Your team gather surveys to customers regarding their ticketing system's performance. Your role in the team is to gather reports regarding customer satisfaction and sentiment analysis into a single dashboard and present your insight.

Analyze the following metrics and other insight you can find in the dataset:

- Survey response rate
- Customer Satisfaction score (CSAT)
- Customer Effort Score (CES)
- Net Promoter Score (NPS)
- Sentiment Analysis



## Data Preparation

In [1]:
import numpy as np
import pandas as pd
import os

pd.options.display.max_columns = 999
pd.options.display.float_format = "{:.2f}".format

### Access to Drive

Write where you put the data in google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Where is your data_path


In [2]:
data = pd.read_csv('/content/assignment_ticket_system_review.csv')

### Read Data

Read the file **assignment_ticket_system_review.csv**

In [8]:
# Read Data
data.head()

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text
0,T_02161,2024-11-20,Zendesk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T_00229,2024-10-06,Zendesk,3.00,4.00,3.00,3.00,2.00,6.00,Customer tickets managements
2,T_04527,2024-12-26,Zoho Desk,5.00,5.00,5.00,5.00,5.00,8.00,"After 6 months of using the Zoho desk, we shif..."
3,T_03190,2024-12-08,Zoho Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T_00644,2024-10-17,Zendesk,5.00,3.00,4.00,5.00,5.00,6.00,Pros:Zendesk has always been one of the go-to ...


The following is the dictionary for the data, survey is only valid if all of the survey questions and text review is not blank (null):

**General Information**
- id_survey: identifier for each survey
- date_of_survey: date of survey taken
- ticket_system: The name of the ticket system being reviewed (e.g. Zoho Desk)

**Survey Questions**
- overall_rating: The overall satisfaction rating given by the reviewer, ranging from 1 to 5
- customer_service: The satisfaction rating for the customer service provided by the ticket system, ranging from 1 to 5.
- features: The satisfaction rating for the features of the ticket system, ranging from 1 to 5
- value_for_money: The satisfaction rating for the value for money provided by the ticket system, ranging from 1 to 5
- ease_of_use: The rating for how easy the ticket system is to use, ranging from 1 to 5
- likelihood_to_recommend: The likelihood that the reviewer would recommend the ticket system to others, ranging from 1 to 10
- overall_text: The full text of the overall review, providing detailed feedback on the ticket system.


In [5]:
# Check the type of data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_survey                1462 non-null   object 
 1   date_of_survey           1462 non-null   object 
 2   ticket_system            1462 non-null   object 
 3   overall_rating           787 non-null    float64
 4   customer_service         787 non-null    float64
 5   features                 787 non-null    float64
 6   value_for_money          787 non-null    float64
 7   ease_of_use              787 non-null    float64
 8   likelihood_to_recommend  787 non-null    float64
 9   overall_text             787 non-null    object 
dtypes: float64(6), object(4)
memory usage: 114.3+ KB


### Data Cleansing

Convert the date column into proper date_time format.

In [7]:
# Convert data type
data['date_of_survey'] = pd.to_datetime(data['date_of_survey'], format='%Y-%m-%d')

In [9]:
# Read Data
data.head()

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text
0,T_02161,2024-11-20,Zendesk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T_00229,2024-10-06,Zendesk,3.00,4.00,3.00,3.00,2.00,6.00,Customer tickets managements
2,T_04527,2024-12-26,Zoho Desk,5.00,5.00,5.00,5.00,5.00,8.00,"After 6 months of using the Zoho desk, we shif..."
3,T_03190,2024-12-08,Zoho Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T_00644,2024-10-17,Zendesk,5.00,3.00,4.00,5.00,5.00,6.00,Pros:Zendesk has always been one of the go-to ...


## Survey Analysis

### Response Rate

Start by analyzing how many customers has filled the survey, indicated by whether the overall_rating is not blank.

In [16]:
# How many customer responded to the survey?
responded_customers_count = data[data['overall_rating'].notna()].shape[0]

print(f"Jumlah pelanggan yang mengisi survei: {num_customers_responded}")

Jumlah pelanggan yang mengisi survei: 787


Create a new dataframe that consists only of those who have responded the survey to simplify calculating the CSAT, CES, and NPS Score.

In [18]:
# Responded Customer
responded_customers_df = data[data['overall_rating'].notna()]
responded_customers_df.head()

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text
1,T_00229,2024-10-06,Zendesk,3.00,4.00,3.00,3.00,2.00,6.00,Customer tickets managements
2,T_04527,2024-12-26,Zoho Desk,5.00,5.00,5.00,5.00,5.00,8.00,"After 6 months of using the Zoho desk, we shif..."
4,T_00644,2024-10-17,Zendesk,5.00,3.00,4.00,5.00,5.00,6.00,Pros:Zendesk has always been one of the go-to ...
6,T_04682,2024-12-28,Zoho Desk,5.00,4.00,5.00,5.00,5.00,8.00,It has been very useful so far to integrate mu...
8,T_01238,2024-11-02,Freshdesk,4.00,4.00,4.00,5.00,4.00,8.00,Pros:It's easy to use and very intuitive.We ha...


### CSAT Score

Measure the customer's overall satisfaction score (CSAT) with the following formula:

$$
CSAT = \frac{\Sigma\ total\ satisfaction\ score}{number\ of\ responded\ customer \times \max\ rating}
$$

The max rating is inserted to convert the CSAT score into percentage.

CSAT score can be classified into categories based on the result. There is no absolute threshold for each categories but the following is the common threshold:

- \>= 90%: Excellent
- 75%-90%: Good
- 60-75%: Fair
- \<60%: Poor

In [19]:
# CSAT Score
# Calculate CSAT Score
max_rating = 5  # As given in the dataset, the max rating is 5

# Sum of all satisfaction scores (i.e., overall_rating)
total_satisfaction_score = responded_customers_df['overall_rating'].sum()

# Number of responded customers
responded_customers_count = responded_customers_df.shape[0]

# Calculate CSAT as percentage
CSAT_score = (total_satisfaction_score / (responded_customers_count * max_rating)) * 100

# Categorize CSAT Score
if CSAT_score >= 90:
    CSAT_category = 'Excellent'
elif 75 <= CSAT_score < 90:
    CSAT_category = 'Good'
elif 60 <= CSAT_score < 75:
    CSAT_category = 'Fair'
else:
    CSAT_category = 'Poor'

# Output the CSAT Score and category
print(f"CSAT Score: {CSAT_score:.2f}%")
print(f"CSAT Category: {CSAT_category}")


CSAT Score: 91.18%
CSAT Category: Excellent


Measure the satisfaction score for the following attributes:

- customer service
- features
- value for money

In [21]:
# Satisfaction Score for Attributes
# Define the attributes to calculate satisfaction scores for
attributes = ['customer_service', 'features', 'value_for_money']

# Initialize a dictionary to store the satisfaction scores for each attribute
satisfaction_scores = {}

# Calculate satisfaction score for each attribute
for attribute in attributes:
    # Sum of all ratings for the current attribute
    total_score = responded_customers_df[attribute].sum()

    # Calculate satisfaction score as a percentage
    satisfaction_score = (total_score / (responded_customers_count * max_rating)) * 100

    # Store the result in the dictionary
    satisfaction_scores[attribute] = satisfaction_score

# Output the satisfaction scores for the attributes
for attribute, score in satisfaction_scores.items():
    print(f"{attribute.replace('_', ' ').title()} Satisfaction Score: {score:.2f}%")


Customer Service Satisfaction Score: 67.34%
Features Satisfaction Score: 88.34%
Value For Money Satisfaction Score: 87.60%


### CES Score

Measure CES with the following formula


$$
CES = \frac{\Sigma\ total\ effort\ score}{number\ of\ responded\ customer \times \max\ rating}
$$

In [22]:
# CES Score
# Assuming 'customer_service' is used as an indicator of customer effort in the dataset
# If there's another column for effort, replace 'customer_service' with the correct column name

# Calculate CES (Customer Effort Score)
effort_column = 'customer_service'  # Replace with the relevant column if different

# Sum of all effort scores (i.e., customer_service or a specific effort-related column)
total_effort_score = responded_customers_df[effort_column].sum()

# Calculate CES as percentage
CES_score = (total_effort_score / (responded_customers_count * max_rating)) * 100

# Output the CES Score
print(f"CES Score: {CES_score:.2f}%")



CES Score: 67.34%


### NPS Score

To calculate the NPS score, first we must convert the **would_you_recommend** column into proper NPS Category based on the rating value:

* Promoter: Rating 9-10
* Passive: Rating 7-8
* Detractor: Rating < 7

In [26]:
# Ganti 'likelihood_to_recommend' dengan nama kolom yang benar
nps_column = 'likelihood_to_recommend'

# Create a new column to categorize respondents into Promoters, Passives, and Detractors
responded_customers_df['nps_category'] = responded_customers_df[nps_column].apply(
    lambda x: 'Promoter' if x >= 9 else ('Passive' if x >= 7 else 'Detractor')
)

# Calculate the percentage of Promoters and Detractors
promoters_count = responded_customers_df[responded_customers_df['nps_category'] == 'Promoter'].shape[0]
detractors_count = responded_customers_df[responded_customers_df['nps_category'] == 'Detractor'].shape[0]

# Calculate the total number of respondents
total_respondents = responded_customers_df.shape[0]

# Calculate the percentages
percentage_promoters = (promoters_count / total_respondents) * 100
percentage_detractors = (detractors_count / total_respondents) * 100

# Calculate NPS Score
NPS_score = percentage_promoters - percentage_detractors

# Output the NPS categories and NPS score
print(f"Percentage of Promoters: {percentage_promoters:.2f}%")
print(f"Percentage of Detractors: {percentage_detractors:.2f}%")
print(f"NPS Score: {NPS_score:.2f}")


Percentage of Promoters: 31.89%
Percentage of Detractors: 19.95%
NPS Score: 11.94


<ipython-input-26-ae79a571aeac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responded_customers_df['nps_category'] = responded_customers_df[nps_column].apply(



Calculate the NPS Score with the following formula

$$
NPS = \frac{Promoter - Detractor}{Total\ Survey\ Responded}
$$

In [28]:
# NPS Score
# Calculate NPS Score using the new formula
promoters_count = responded_customers_df[responded_customers_df['nps_category'] == 'Promoter'].shape[0]
detractors_count = responded_customers_df[responded_customers_df['nps_category'] == 'Detractor'].shape[0]

# Calculate the total number of respondents
total_respondents = responded_customers_df.shape[0]

# Calculate NPS Score using the new formula
NPS_score = ((promoters_count - detractors_count) / total_respondents) * 100

# Output the NPS Score
print(f"NPS Score: {NPS_score:.2f}")


NPS Score: 11.94


NPS score can be ranging from -100 (when all customers are detractor) to 100 (when all customers are promoter).

NPS Score can be classified into categories based on the following threshold:

- \>= 70: Excellent
- 50-69: Very Good
- 30-49: Good
- 0-29: Average
- \< 0: Poor

## Sentiment Analysis

Create a new dataframe with no blank overall_text.

In [29]:
# Create new dataframe
# Filter dataframe to create a new one with no blank 'overall_text'
non_blank_reviews_df = responded_customers_df[responded_customers_df['overall_text'].notna()]

# Optionally, reset the index of the new DataFrame
non_blank_reviews_df = non_blank_reviews_df.reset_index(drop=True)

# Output the first few rows to verify
print(non_blank_reviews_df.head())


  id_survey date_of_survey ticket_system  overall_rating  customer_service  \
0   T_00229     2024-10-06       Zendesk            3.00              4.00   
1   T_04527     2024-12-26     Zoho Desk            5.00              5.00   
2   T_00644     2024-10-17       Zendesk            5.00              3.00   
3   T_04682     2024-12-28     Zoho Desk            5.00              4.00   
4   T_01238     2024-11-02     Freshdesk            4.00              4.00   

   features  value_for_money  ease_of_use  likelihood_to_recommend  \
0      3.00             3.00         2.00                     6.00   
1      5.00             5.00         5.00                     8.00   
2      4.00             5.00         5.00                     6.00   
3      5.00             5.00         5.00                     8.00   
4      4.00             5.00         4.00                     8.00   

                                        overall_text nps_category  
0                       Customer tickets m

### Text Cleansing

In order to get more accurate sentiment, several text cleansing need to be done. However, in most of recent sentiment analysis models and algorithm, the only text cleansing needed are as follows:

* Clean double whitespace
* Clean URL/website
* Clean username (mostly in social media or digital text)

In [30]:
import re

def cleansing_text(x):
  # clean double whitespace
  out_text = ' '.join(x.split())

  # clean url
  out_text = re.sub(r"http\S+|www\S+|https\S+", 'http', out_text)

  # clean username
  out_text = re.sub(r"@\S+", '@user', out_text)

  return(out_text)

cleansing_text(" Doesn't  dissapoint. The car       was great. It was the best car rental experiences I've had! Salute to @jone who recommend https:/rental.com")

"Doesn't dissapoint. The car was great. It was the best car rental experiences I've had! Salute to @user who recommend http"

In [32]:
# apply cleansing to review
import re

# Define the text cleansing function
def cleansing_text(x):
    # Clean double whitespace
    out_text = ' '.join(x.split())

    # Clean URL
    out_text = re.sub(r"http\S+|www\S+|https\S+", 'http', out_text)

    # Clean username
    out_text = re.sub(r"@\S+", '@user', out_text)

    return out_text

# Apply the cleansing function to the 'overall_text' column
non_blank_reviews_df['cleaned_text'] = non_blank_reviews_df['overall_text'].apply(cleansing_text)

# Output the first few rows to verify the changes
print(non_blank_reviews_df[['overall_text', 'cleaned_text']].head())


                                        overall_text  \
0                       Customer tickets managements   
1  After 6 months of using the Zoho desk, we shif...   
2  Pros:Zendesk has always been one of the go-to ...   
3  It has been very useful so far to integrate mu...   
4  Pros:It's easy to use and very intuitive.We ha...   

                                        cleaned_text  
0                       Customer tickets managements  
1  After 6 months of using the Zoho desk, we shif...  
2  Pros:Zendesk has always been one of the go-to ...  
3  It has been very useful so far to integrate mu...  
4  Pros:It's easy to use and very intuitive.We ha...  


### Sentiment Analysis

Create a sentiment categories using algorithm of your own choice.

In [34]:
# Sentiment Algorithm
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Pastikan Anda sudah mengunduh resource untuk VADER jika belum
nltk.download('vader_lexicon')

# Inisialisasi SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Fungsi untuk menentukan kategori sentimen berdasarkan skor VADER
def sentiment_category(text):
    # Mendapatkan skor sentimen
    sentiment_score = sia.polarity_scores(text)['compound']

    # Kategorikan berdasarkan nilai sentimen
    if sentiment_score >= 0.05:
        return 'Positive'
    elif sentiment_score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply the sentiment function to the cleaned_text column
non_blank_reviews_df['sentiment_category'] = non_blank_reviews_df['cleaned_text'].apply(sentiment_category)

# Output the first few rows to verify the sentiment categorization
print(non_blank_reviews_df[['cleaned_text', 'sentiment_category']].head())


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


                                        cleaned_text sentiment_category
0                       Customer tickets managements            Neutral
1  After 6 months of using the Zoho desk, we shif...           Positive
2  Pros:Zendesk has always been one of the go-to ...           Positive
3  It has been very useful so far to integrate mu...           Positive
4  Pros:It's easy to use and very intuitive.We ha...           Positive


Check the number of data by sentiment.

In [36]:
# Number of Sentiment
# Check the number of data by sentiment category
sentiment_counts = non_blank_reviews_df['sentiment_category'].value_counts()

# Output the sentiment counts
print(sentiment_counts)


sentiment_category
Positive    673
Neutral      86
Negative     28
Name: count, dtype: int64


## Finalize Data for Reporting

Save the review data with NPS category and sentiment information to new csv for the dashboard.

In [39]:
# Save Data
# Select relevant columns to save for the dashboard
final_df = non_blank_reviews_df[['id_survey', 'date_of_survey', 'ticket_system', 'overall_rating',
                                  'customer_service', 'features', 'value_for_money', 'ease_of_use',
                                  'likelihood_to_recommend', 'overall_text', 'nps_category',
                                  'sentiment_category']]

# Save the final DataFrame to a new CSV file with the specified name
final_df.to_csv('final_ticket_system_review.csv', index=False)

# Output a confirmation message
print("Data has been saved to 'final_ticket_system_review.csv'")


Data has been saved to 'final_ticket_system_review.csv'
